In [111]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB
import pandas as pd

db = DB()

In [113]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz']

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp"
})

In [114]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['ballad'],
 ['pop', 'jazz'],
 ['jazz', 'ballad'],
 ['hip-hop'],
 ['ballad', 'jazz'],
 ['hip-hop'],
 ['pop', 'hip-hop'],
 ['hip-hop', 'pop'],
 ['k-pop'],
 ['pop']]

In [115]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623d861aacf38dfa6647bd5c', '623d861aacf38dfa6647bd5d', '623d861aacf38dfa6647bd5e', '623d861aacf38dfa6647bd5f', '623d861bacf38dfa6647bd60', '623d861bacf38dfa6647bd61', '623d861bacf38dfa6647bd62', '623d861bacf38dfa6647bd63', '623d861bacf38dfa6647bd64', '623d861bacf38dfa6647bd65']


,id,name,artists,artists_name
0,3UPjb91Fwm7u2tAm92Bk0p,why break up?,02SENF0XVfxRTZrniEV2aF,Sin Ye Young


In [116]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
    db.observe_seed_zone(features)
    
print(genre_seed_list)
features_list[0]

[      count
k-pop     1,      count
pop      1
jazz     1,       count
jazz      1
k-pop     1,       count
k-pop     1,       count
k-pop     1
jazz      1,       count
k-pop     1,       count
pop       1
k-pop     1,       count
k-pop     1
pop       1,       count
k-pop     1,     count
pop     1]


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3UPjb91Fwm7u2tAm92Bk0p,0.554,0.526,7,-5.21,0.0394,0.808,0,0.074,0.23,129.845


In [117]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

In [118]:
reco_list[0].head()

,id,name,artists,artists_name
0,3ZlxwbbbmEqRZsOjl06Qii,Waiting for Spring,6pU8o91xAS0aWNjj06nQSU,Kassy
1,3ImaACMfZ1YdTrkVkPGVAR,Break Up In The Morning,7oFIkpNpLrTBgZW6w55W1J,4MEN
2,4so85rrFc5TCgT6rd2vweV,서툰 이별을 하려해,13PsdrWmNx5EcibArzkbcO,Yountoven
3,4PVlgqnAacGP0ppxbgkvzZ,Say Goodbye,4dvmLXorXADxVq8EcySEWk,Kim Na Young
4,6EWWZTKKhFh7sTYsm65zfV,0310,6dhfy4ByARPJdPtMyrUYJK,Yerin Baek


In [119]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3ZlxwbbbmEqRZsOjl06Qii,0.501,0.533,7,-4.844,0.0329,0.846,0.000000,0.1230,0.196,139.619
1,3ImaACMfZ1YdTrkVkPGVAR,0.510,0.475,7,-6.703,0.0319,0.739,0.000000,0.1210,0.218,131.665
2,4so85rrFc5TCgT6rd2vweV,0.538,0.470,7,-7.111,0.0378,0.820,0.000000,0.2430,0.196,133.585
3,4PVlgqnAacGP0ppxbgkvzZ,0.466,0.481,7,-4.142,0.0285,0.610,0.000000,0.1210,0.195,133.684
4,6EWWZTKKhFh7sTYsm65zfV,0.548,0.459,7,-6.467,0.0304,0.703,0.000015,0.0989,0.525,135.975


In [1]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

NameError: name 'features_list' is not defined

In [121]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

-- 라운드 10
1 round : ECV 15.11%
2 round : ECV 43.77%
3 round : ECV 49.13%
4 round : ECV 50.51%
5 round : ECV 50.89%
6 round : ECV 51.04%
7 round : ECV 51.04%
8 round : ECV 51.04%
9 round : ECV 51.04%
10 round : ECV 51.04%
Clustering End.


In [122]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
1,3ImaACMfZ1YdTrkVkPGVAR,Break Up In The Morning,7oFIkpNpLrTBgZW6w55W1J,4MEN
4,6EWWZTKKhFh7sTYsm65zfV,0310,6dhfy4ByARPJdPtMyrUYJK,Yerin Baek
6,3LpAsXjqTQ5FUyptb2lujH,The day was beautiful,6pU8o91xAS0aWNjj06nQSU,Kassy
8,488mXgKGA5CM3kn6pAngDZ,Bad Love,64jfAecBriamQmMs0WAKtj,HYNN
9,2wKnGjAnQxH9rFDiTSBTyp,서쪽 하늘,5Ecf8RR2UWFAvyeItabffF,Lee Seung Chul


In [123]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(623d861aacf38dfa6647bd5c)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd6b)가 등록되었습니다.
우체통(623d861aacf38dfa6647bd5d)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd6c)가 등록되었습니다.
우체통(623d861aacf38dfa6647bd5e)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd6d)가 등록되었습니다.
우체통(623d861aacf38dfa6647bd5f)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd6e)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd60)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd6f)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd61)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd70)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd62)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd71)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd63)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd72)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd64)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd73)가 등록되었습니다.
우체통(623d861bacf38dfa6647bd65)에
추천 플레이리스트 MuLetter(623d8630acf38dfa6647bd74)가 등록되었습니다.


### reply

In [163]:
import math as mt

In [164]:
track_list = list()
reco_track_list = list()

for mail_box_id in mail_boxes_list:
    tracks = db.get_mail_box(mail_box_id)
    reco_tracks = db.get_mail(mail_box_id)

    track_list.append(tracks)
    reco_track_list.append(reco_tracks)
    
track_list[0].head()

,id,name,artists,artists_name
0,3UPjb91Fwm7u2tAm92Bk0p,why break up?,02SENF0XVfxRTZrniEV2aF,Sin Ye Young
1,0RRJ2PHFSCyBaaydDHOv1u,Hello,2TCV57G47ttyy1JsVyucXg,Sohyang
2,3LpAsXjqTQ5FUyptb2lujH,The day was beautiful,6pU8o91xAS0aWNjj06nQSU,Kassy
3,7hSxOrxogaR7YGpg4l8wmd,I will be your shining star,4whjJjlywO81WrDptpdttm,Song I Han
4,4rMEoXVYww6Xy7q9wSj4gy,Love Always Run Away,75MOYjGEyyH5U4ZFHOPvxR,Lim Young Woong


In [174]:
sel_track_list = list()

for idx, reco_tracks in enumerate(reco_track_list):
    mail_box_id = mail_boxes_list[idx]
    n = mt.floor(len(reco_tracks) * 0.075)
    n = 1 if n < 1 else n
    
    sel_tracks = reco_tracks.sample(n=n)
    db.new_mail_box_seed(mail_box_id, sel_tracks)

    sel_track_list.append(sel_tracks)

In [202]:
# 1. Get Token
token = spotify.get_token()

_sel_track_list = list()
features_list = list()
recos_list = list()
reco_features_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    tracks = track_list[idx]
    _sel_tracks = pd.concat([tracks,sel_tracks], ignore_index=True)
    genres = spotify.get_genres(token, _sel_tracks)

    og_features = spotify.get_features(token, tracks)
    reply_features = spotify.get_features(token, sel_tracks)
    db.observe_seed_zone(reply_features)

    recos = spotify.get_recommend(sel_tracks, reply_features, genres, token,
                                 og=tracks)
    
    if str(type(recos)) == "<class 'requests.models.Response'>":
        print(recos.url)
        print(idx,recos.json())
        
    reco_features = spotify.get_features(token, recos)
    features = pd.concat([og_features, reply_features], ignore_index=True)
    
    features_list.append(features)
    recos_list.append(recos)
    reco_features_list.append(reco_features)
    _sel_track_list.append(_sel_tracks)

In [203]:
norm_features_list = list()

for idx, reco_features in enumerate(reco_features_list):
    features = features_list[idx]
    norm_features = dp.make_norm(features, reco_features)

    norm_features_list.append(norm_features)

In [204]:
filterings = list()
kmeans_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print("{}번 박스".format(idx))
    norm_features = norm_features_list[idx]
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=5)
    clear_output(wait=True)

    _filtering_music_list = dp.music_filtering(sel_tracks, kmeans)

    if len(_filtering_music_list) <= (100 + len(sel_tracks)):
        filterings.append(_filtering_music_list)
        kmeans_list.append(kmeans)
        continue
    else:
        filter_music = norm_features.set_index("id").loc[_filtering_music_list].reset_index()
    while True:
        kmeans = utils.KMeans(
            datas=filter_music
        )
        kmeans.run(early_stop_cnt=5)
        clear_output(wait=True)


        _filtering_music_list = dp.music_filtering(sel_tracks, kmeans)

        if len(_filtering_music_list) <= (100 + len(sel_tracks)):
            filterings.append(_filtering_music_list)
            kmeans_list.append(kmeans)
            break
        else:
            filter_music = norm_features.set_index("id").loc[_filtering_music_list].reset_index()

9번 박스
1 round : ECV 24.18%
2 round : ECV 41.28%
3 round : ECV 42.93%
4 round : ECV 44.77%
5 round : ECV 45.58%
6 round : ECV 45.94%
7 round : ECV 46.05%
8 round : ECV 46.05%
9 round : ECV 46.05%
10 round : ECV 46.05%
11 round : ECV 46.05%
12 round : ECV 46.05%
13 round : ECV 46.05%
Clustering End.


In [205]:
reco_musics = [_ in filterings[1] for _ in recos_list[1]['id']]
recos_list[1][reco_musics]

,id,name,artists,artists_name
7,74cq6AFbsKdUVZMpG2TPEr,Dinosaur,6LqNN22kT3074XbTVUrhzX,Kesha
11,5xvUgoVED1F4mBu8FL0HaW,Ready or Not,4VhL8KLjVso4vLfOLVViTb,Bridgit Mendler
15,3WKz5JDH0St3Smips7NlOM,Grudges,74XFHRwlV6OrjEM0A2NCMF,Paramore
19,2VEsmoek0sol9MnJFyoG9e,Burnin' Up,7gOdHgIoIKoe4i9Tta6qdD,Jonas Brothers
21,2ZKvBNoOjYzbB00EwutMgQ,Boogie Feet (feat. Eagles of Death Metal),6LqNN22kT3074XbTVUrhzX,Kesha
22,6sqNctd7MlJoKDOxPVCAvU,My Happy Ending,0p4nmQO2msCgU4IF37Wi3j,Avril Lavigne
23,77o1HdXHm1hLd7Ebe9ygd0,THAT BITCH,1o2NpYGqHiCq7FoiYdyd1x,Bea Miller
34,1r3myKmjWoOqRip99CmSj1,Don't Wanna Go Home,07YZf4WDAMNwqr4jfgOZ8y,Jason Derulo
36,13plQdOoWSSXPRUSZc5FuM,I Know You Want Me (Calle Ocho),0TnOYISbd1XYRBk9myaseg,Pitbull
40,2zQl59dZMzwhrmeSBEgiXY,Watermelon Man,2ZvrvbQNrHKwjT7qfGFFUW,Herbie Hancock


In [206]:
for idx,mail_box_id in enumerate(mail_boxes_list):
    recos = recos_list[idx]
    filtering = filterings[idx]
    reco_musics = [_ in filtering for _ in recos['id']]
    
    db.new_mail(mail_box_id,
               recos[reco_musics])